In [1]:
from lib.data_util import read_data
from lib.documentation import build_description_table

### Pre-process data

In [2]:
df = read_data("data/statcast_pitch_swing_data_20240402_20240630.arrow")
build_description_table(df, "glossary/data_definitions.md", "glossary/data_dictionary.md")

In [3]:
df

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,…,release_extension,game_pk,pitcher_1,fielder_2_1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
cat,date,f64,f64,f64,cat,u32,u32,cat,cat,u8,str,cat,cat,cat,cat,cat,u8,cat,u8,u8,f64,f64,f64,f64,u32,u32,u32,u8,u8,cat,f64,f64,u32,f64,f64,f64,…,f64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,f64,f64,f64,f64,u8,u8,u8,u8,u8,u8,cat,u8,u8,u8,u8,u8,u8,u8,u8,cat,cat,u16,f64,f64,f64,f64
"""FF""",2024-04-02,95.0,-2.01,5.22,"""Rocchio, Brayan""",677587,622491,"""single""","""hit_into_play""",3,"""Brayan Rocchio singles on a po…","""L""","""R""","""SEA""","""CLE""","""X""",7,"""popup""",0,2,-0.85,1.2,0.69,3.18,null,null,666310,2,6,"""Top""",87.46,125.97,663728,8.809589,-138.184461,-2.337745,…,5.4,745273,622491,663728,664034,593871,668942,641487,686527,677594,571745,55.11,0.654,0.59,0.9,1,1,0,4,46,3,"""4-Seam Fastball""",0,4,4,0,4,0,4,0,"""Infield shade""","""Standard""",239,-0.006,0.388,null,null
"""CH""",2024-04-02,88.5,-2.09,4.95,"""Rocchio, Brayan""",677587,622491,"""""","""foul""",4,"""Brayan Rocchio singles on a po…","""L""","""R""","""SEA""","""CLE""","""S""",null,"""""",0,1,-1.4,0.07,-0.3,2.29,null,null,666310,2,6,"""Top""",null,null,663728,7.152185,-128.763652,-0.628007,…,5.4,745273,622491,663728,664034,593871,668942,641487,686527,677594,571745,55.06,null,null,null,null,null,null,null,46,2,"""Changeup""",0,4,4,0,4,0,4,0,"""Standard""","""Standard""",253,0.0,-0.045,null,null
"""SI""",2024-04-02,95.0,-2.02,5.12,"""Rocchio, Brayan""",677587,622491,"""""","""called_strike""",2,"""Brayan Rocchio singles on a po…","""L""","""R""","""SEA""","""CLE""","""S""",null,"""""",0,0,-1.48,0.15,-0.14,3.15,null,null,666310,2,6,"""Top""",null,null,663728,8.099205,-138.195334,0.123694,…,5.5,745273,622491,663728,664034,593871,668942,641487,686527,677594,571745,55.04,null,null,null,null,null,null,null,46,1,"""Sinker""",0,4,4,0,4,0,4,0,"""Standard""","""Standard""",238,0.0,-0.042,null,null
"""SI""",2024-04-02,90.7,-1.26,5.13,"""Ohtani, Shohei""",660271,657277,"""walk""","""ball""",11,"""Shohei Ohtani walks. Enrique…","""L""","""R""","""LAD""","""SF""","""B""",null,"""""",3,0,-1.45,0.09,-2.25,3.29,null,null,571771,2,4,"""Bot""",null,null,672275,0.6209,-132.024381,1.22685,…,6.8,746169,657277,672275,527038,642731,656305,605113,624424,808982,573262,53.68,null,0.691353,0.7,1,0,0,null,37,4,"""Sinker""",5,2,5,2,2,5,5,2,"""Infield shade""","""Standard""",234,0.009,0.082,null,null
"""FF""",2024-04-02,95.4,-1.95,5.12,"""Hedges, Austin""",595978,622491,"""strikeout""","""foul_tip""",5,"""Austin Hedges strikes out on a…","""R""","""R""","""SEA""","""CLE""","""S""",2,"""""",1,2,-1.08,0.9,-0.04,2.52,null,null,666310,1,6,"""Top""",null,null,663728,7.315465,-138.777335,-3.229775,…,5.5,745273,622491,663728,664034,593871,668942,641487,686527,677594,571745,55.04,null,0.0,0.0,1,0,0,null,45,4,"""4-Seam Fastball""",0,4,4,0,4,0,4,0,"""Standard""","""Standard""",238,0.008,-0.215,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SL""",2024-06-30,85.9,1.63,5.68,"""Cave, Jake""",595909,694363,"""""","""called_strike""",4,"""Jake Cave grounds out, second …","""L""","""L""","""CWS""","""COL""","""S""",null,"""""",0,0,-0.14,0.65,-0.46,2.43,null,669911,663898,2,10,"""Top""